In [59]:
import requests
from bs4 import BeautifulSoup

url = "https://store.steampowered.com/search/?tags=19&supportedlang=english&filter=topsellers&ndl=1" 
#Action "https://store.steampowered.com/search/?tags=19&supportedlang=english&filter=topsellers&ndl=1" 
#SinglePlayer https://store.steampowered.com/search/?filter=topsellers&tags=4182&supportedlang=english&ndl=1
# Adventure https://store.steampowered.com/search/?filter=topsellers&tags=21&supportedlang=english&ndl=1
#Indie https://store.steampowered.com/search/?filter=topsellers&tags=492&supportedlang=english&ndl=1
#Simulation https://store.steampowered.com/search/?filter=topsellers&tags=599&supportedlang=english&ndl=1
#Strategy https://store.steampowered.com/search/?filter=topsellers&tags=9&supportedlang=english&ndl=1
#Casual https://store.steampowered.com/search/?filter=topsellers&tags=597&supportedlang=english&ndl=1
#RPG https://store.steampowered.com/search/?filter=topsellers&tags=122&supportedlang=english&ndl=1
#Multiplayer https://store.steampowered.com/search/?filter=topsellers&tags=3859&supportedlang=english&ndl=1
#Atmospheric https://store.steampowered.com/search/?filter=topsellers&tags=4166&supportedlang=english&ndl=1
#2d https://store.steampowered.com/search/?filter=topsellers&tags=3871&supportedlang=english&ndl=1 
#story rich https://store.steampowered.com/search/?filter=topsellers&tags=1742&supportedlang=english&ndl=1
 #co-op https://store.steampowered.com/search/?filter=topsellers&tags=1685&supportedlang=english&ndl=1
#first-person https://store.steampowered.com/search/?filter=topsellers&tags=3839&supportedlang=english&ndl=1
#3d https://store.steampowered.com/search/?filter=topsellers&tags=4191&supportedlang=english&ndl=1
#open world https://store.steampowered.com/search/?filter=topsellers&tags=1695&supportedlang=english&ndl=1

response = requests.get(url)
# , headers={
#     'cookie': 'steamCountry=KR!13817a80e36b14f1a94dd77fb2142a64; browserid=3999848730290483524; sessionid=2225f1149bbcf81b9631038f; timezoneOffset=32400,0; app_impressions=2344520@1_7_7_7000_150_1|2138330@1_7_7_7000_150_1|2358720@1_7_7_7000_150_1|2074920@1_7_7_7000_150_1|2183900@1_7_7_7000_150_1|1245620@1_7_7_7000_150_1|2139460@1_7_7_7000_150_1|1049590@1_7_7_7000_150_1|1091500@1_7_7_7000_150_1|578080@1_7_7_7000_150_1',
#     'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
#     'accept-language':'ko-KR,ko;q=0.9',
#     'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
#     'accept-encoding': 'gzip, deflate, br, zstd',
#     'cache-control':'no-cache'
# }

soup = BeautifulSoup(response.content, 'html.parser')

app_ids = []

container = soup.find('div', id='search_result_container')

if container:
    for item in container.find_all('a'):
        link =item['href']
        if 'app' in link:
            app_id = link.split('app')[1].split('/')[1]
            app_ids.append(app_id)

app_id=app_ids[:10]
app_id

['578080',
 '1049590',
 '1091500',
 '2139460',
 '1245620',
 '2074920',
 '2183900',
 '2358720',
 '2344520',
 '2138330']

In [74]:
import requests
import pandas as pd

# Steam Store API에서 특정 게임의 세부 정보를 가져오는 함수
def get_game_details(app_id):
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]["success"]:
            game_data = data[str(app_id)]["data"]
            return {
                "app_id": app_id,
                "name": game_data.get("name"),
                "genres": ", ".join([genre['description'] for genre in game_data.get("genres", [])]),
                "header_image": game_data.get("header_image"),
            }
    return {}

# Steam API를 통해 리뷰 데이터를 가져오는 함수
def get_reviews_for_game(app_id, num_reviews=100):
    url = f"https://store.steampowered.com/appreviews/{app_id}"
    params = {
        'json': 1,
        'num_per_page': num_reviews,  # 한 번에 가져올 리뷰 수
        'filter': 'recent',  # 최신 리뷰를 가져옴
        'language': 'all'  # 모든 언어로 된 리뷰를 가져옴
    }
    response = requests.get(url, params=params)
    data = response.json()

    reviews = []

    if 'reviews' in data:
        for review in data['reviews']:
            reviews.append({
                'review_text': review.get('review'),
                'review_score': review.get('voted_up'),
                'language': review.get('language')
            })

    return reviews

# 게임 이름, 장르, 이미지, 사용자 평가 및 언어를 포함한 데이터를 DataFrame으로 정리하는 함수
def get_game_data_with_reviews(app_ids, num_reviews=100):
    all_data = []

    for app_id in app_ids:
        # 1. 게임 세부 정보 가져오기
        game_info = get_game_details(app_id)
        
        if game_info:
            # 2. 게임에 대한 리뷰 가져오기
            reviews = get_reviews_for_game(app_id, num_reviews)
            
            # 3. 각 리뷰에 게임 세부 정보 추가
            for review in reviews:
                all_data.append({
                    'app_id': game_info['app_id'],
                    'game_name': game_info['name'],
                    'game_genres': game_info['genres'],
                    'game_image': game_info['header_image'],
                    'review_text': review['review_text'],
                    'review_score': review['review_score'],
                    'language': review['language']
                })

    # 모든 데이터를 DataFrame으로 변환
    df = pd.DataFrame(all_data)
    return df

# 게임 데이터 및 리뷰 가져오기
df_game_data = get_game_data_with_reviews(app_id, num_reviews=50)  # 각 게임당 50개의 리뷰 가져오기

# CSV 파일로 저장
df_game_data.to_csv('action.csv', index=False)


In [75]:
ex=pd.read_csv('action.csv')
ex['app_id'].value_counts()

app_id
578080     50
1049590    50
1091500    50
1245620    50
2183900    50
2358720    50
2344520    50
2138330    50
Name: count, dtype: int64

In [ ]:
['578080', # 1
 '1049590', # 2
 '1091500', # 3
 '2139460', # 4 X
 '1245620', # 5
 '2074920', # 6 X 
 '2183900', # 7
 '2358720', # 8
 '2344520', # 9
 '2138330'] # 10

In [91]:
def get_game_data(app_id): # 이미지, id , 이름, 장르 정상적으로 작동 
    url = f"https://store.steampowered.com/api/appdetails?appids={app_id}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data[str(app_id)]["success"]:
            game_data = data[str(app_id)]["data"]
            return {
                "app_id": app_id,
                "name": game_data.get("name"),
                "genres": ", ".join([genre['description'] for genre in game_data.get("genres", [])]),
                "header_image": game_data.get("header_image"),
            }
    return None  # 응답이 성공적이지 않은 경우 None 반환

# 사용 예시
game_info = get_game_data(2074920)
print(game_info)
       


{'app_id': 2074920, 'name': 'The First Descendant', 'genres': 'Action, Adventure, RPG, Free To Play', 'header_image': 'https://shared.akamai.steamstatic.com/store_item_assets/steam/apps/2074920/header.jpg?t=1725356337'}


In [92]:
# Steam API를 통해 리뷰 데이터를 가져오는 함수 # 2139460, 2074920 정상적으로 작동 x 

url = f"https://store.steampowered.com/appreviews/{2074920}"
params = {
        'json': 1,
        'num_per_page': 50,  # 한 번에 가져올 리뷰 수
        'filter': 'recent',  # 최신 리뷰를 가져옴
        'language': 'all'  # 모든 언어로 된 리뷰를 가져옴
}
response = requests.get(url, params=params)
data = response.json()
data
    

{'success': 1,
 'query_summary': {'num_reviews': 0,
  'review_score': 0,
  'review_score_desc': 'No user reviews',
  'total_positive': 0,
  'total_negative': 0,
  'total_reviews': 0},
 'reviews': [],
 'cursor': '*'}